**Соревнование **** 2019 Data Science Bowl **

**Команда:** 
Тимофеев Александр Валентинович (группа 16.Б11-пу): изучение поставленной задачи, подготовка данных и выделение features, выбор модели и ее использование
Глушков Егор Александрович(группа 16.Б12-пу): изучение задачи и предоставляемых файлов, обработка результатов после предсказаний модели, создание необходимого файла submission.csv,оптимизация функций, 
Гаврилова Александра Николаевна (группа 16.Б11-пу): изучение поставленной задачи и определение этапов работы, подготовка данный, определение зависимостей,создание X_train и Y_train для подготовки к обучению
         
**Постановка задачи:** 
По истории участия в образовательной игре необходимо для каждого пользователя определить accuracy group для следующего выполняемого им задания. Существует 5 типов заданий: Bird Measurer, Cart Balancer, Cauldron Filler, Chest Sorter, Mushroom Sorter.

Accuracy groupпринимает 4 возможных значения:

3: задание было решено с первой попытки

2: задание было решенр со второй попытки

1: задание было решено за большее количество попыток

0: задание не было решено


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import reduce

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

1. Считывание данных. Не для всех данных в файле train.csv существовали истинные значения accuracy group, поэтому были убраны строки с теми installation_id, которых нет в train_labels.csv.

In [ ]:
data_train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
print('train', data_train.shape)
data_train_lab = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
print('train_lab', data_train_lab.shape)
data_test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
print('test', data_test.shape)

In [ ]:
new_instal_id = pd.unique(data_train_lab['installation_id'])
print('unique installation id', len(new_instal_id))
reduced_data_train = data_train.loc[data_train['installation_id'].isin(new_instal_id)]
print('reduced data shape', reduced_data_train.shape)

2. Файл train.csv содержит 11 колонок с атрибутами. Не все из них необходимы для создания features. Убираем колонки, которые не будут использоваться для получения features.

In [ ]:
cols_to_drop = ['event_id','timestamp','event_data','event_code','title']
reduced_data_train = reduced_data_train.drop(columns = cols_to_drop)
reduced_data_test = data_test.drop(columns = cols_to_drop)
print(reduced_data_train.head())
reduced_data_train_lab = data_train_lab.drop(columns = ['game_session', 'num_correct', 'num_incorrect', 'accuracy'])

3. Создаем features

Использовалось 5 значений для каждого пользователя: общее время в игры (total_time), общее количество выполненных заданий по разным темам (MAGMAPEAK, CRYSTALCAVES, NONE, TREETOPCITY).

In [ ]:
def get_features(data):
    # какое суммарное время в игре для конкретного игрока
    total_time = data[['installation_id', 'game_time', 'game_session']].groupby(['game_session', 'installation_id'])['game_time'].max().reset_index().drop(columns='game_session')
    total_time = total_time.groupby(['installation_id'])['game_time'].sum().to_frame().rename(columns={"game_time": "all_game_time"})
    total_time['installation_id'] = total_time.index
    total_time = total_time.reset_index(drop=True)
#     print(total_time)
    # какое количество разных активностей в сумме по конкретной теме (world) у конкретного игрока
    activities = data[['installation_id', 'world']].groupby(['installation_id', 'world'])['world'].count()
#     print(activities)
    activity_frame = pd.DataFrame({
        'installation_id': activities.index.get_level_values(0),
        'world': activities.index.get_level_values(1),
        'count': activities.values
                               })
#     print(activity_frame)
    magm = activity_frame[activity_frame['world']=='MAGMAPEAK'].drop(columns=['world'])
    crys = activity_frame[activity_frame['world']=='CRYSTALCAVES'].drop(columns=['world'])
    none = activity_frame[activity_frame['world']=='NONE'].drop(columns=['world'])
    tree = activity_frame[activity_frame['world']=='TREETOPCITY'].drop(columns=['world'])

    total_activities = reduce(lambda  left,right: pd.merge(left,right,on=['installation_id'], how='outer'), [magm, crys, none, tree]).fillna(0).sort_values(by=['installation_id']).reset_index(drop=True)
    total_activities.columns = ['installation_id', 'MAGMAPEAK', 'CRYSTALCAVES', 'NONE', 'TREETOPCITY']
#     print(total_activities)
    # соединим полученные features
    features = pd.merge(total_time, total_activities, on='installation_id', how='outer').drop(columns='installation_id').fillna(0)
    print(features)
    return features

X_train = get_features(reduced_data_train)

4. Создание Y_train

Y_train содержит 5 колонок предсказаний accuracy group для каждого из 5 заданий (Bird Measurer, Cart Balancer, Cauldron Filler, Chest Sorter, Mushroom Sorter) соответсвенно. Предсказывались accuracy group для всех типов заданий, так как заранее не известно, для какого именно задания нужно предсказывать.

In [ ]:
titles = reduced_data_train_lab[['installation_id', 'title', 'accuracy_group']].groupby(['installation_id', 'title'])['accuracy_group'].mean().round(0)
# print(titles)
titles_frame = pd.DataFrame({
    'installation_id': titles.index.get_level_values(0),
    'title': titles.index.get_level_values(1),
    'acc_group': titles.values
                           })
# print(titles_frame)
bm = titles_frame[titles_frame['title']=='Bird Measurer (Assessment)'].drop(columns=['title'])
cb = titles_frame[titles_frame['title']=='Cart Balancer (Assessment)'].drop(columns=['title'])
cf = titles_frame[titles_frame['title']=='Cauldron Filler (Assessment)'].drop(columns=['title'])
cs = titles_frame[titles_frame['title']=='Chest Sorter (Assessment)'].drop(columns=['title'])
ms = titles_frame[titles_frame['title']=='Mushroom Sorter (Assessment)'].drop(columns=['title'])

total_titles = reduce(lambda  left,right: pd.merge(left,right,on=['installation_id'], how='outer'), [bm, cb, cf, cs, ms]).fillna(0).sort_values(by=['installation_id']).reset_index(drop=True)
total_titles.columns = ['installation_id', 'Bird_Measurer', 'Cart_Balancer', 'Cauldron_Filler', 'Chest_Sorter', 'Mushroom_Sorter']

Y_train = total_titles.drop(columns='installation_id')
print(Y_train)

5. Обучение модели

В качестве модели использовался случайный лес в реализации библиотеки scikit-learn.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from joblib import dump, load
forest_model = RandomForestClassifier(random_state=1)

forest_model.fit(X_train, Y_train)
#pickle model to disk
#dump(forest_model, 'my_randomforest_model.joblib') 
print(forest_model)
print(Y_train)


#print(Y_test)
#loading saved model
# forest_model = load('my_randomforest_model.joblib')

6. Были обработаны тестовые данные. Для файла submission.csv было выбрано только одно значение accuracy group из 5 на основе требований задачи.

In [ ]:
X_test = get_features(reduced_data_test)

In [ ]:
Y_test = forest_model.predict(X_test)
print(Y_test)
# print(Y_test.shape, X_test.shape)

Example of Y_test (Bird Measurer, Cart Balancer, Cauldron Filler, Chest Sorter, Mushroom Sorter cols respectively for each installation_id):
* [...
* [1. 1. 3. 0. 3.]
* [0. 0. 0. 0. 0.]
* [0. 0. 3. 0. 0.]
* [1. 3. 2. 0. 3.]
* [0. 0. 0. 0. 3.]
* ... ]


In [ ]:
extracted_test_data = data_test[['installation_id', 'timestamp', 'title']]
max_timestamps = extracted_test_data.groupby(['installation_id'])['timestamp'].max()
last_titles = extracted_test_data[extracted_test_data['timestamp'].isin(max_timestamps)][['installation_id', 'title']].reset_index(drop=True)
print(last_titles)
prediction_groups = []
list_of_titles = list(last_titles['title'])
for i, ttls in enumerate(list_of_titles):
    if ttls == 'Bird Measurer (Assessment)':
        prediction_groups.append(int(Y_test[i][0]))
    elif ttls == 'Cart Balancer (Assessment)':
        prediction_groups.append(int(Y_test[i][1]))
    elif ttls == 'Cauldron Filler (Assessment)':
        prediction_groups.append(int(Y_test[i][2]))
    elif ttls == 'Chest Sorter (Assessment)':
        prediction_groups.append(int(Y_test[i][3]))
    elif ttls == 'Mushroom Sorter (Assessment)':
        prediction_groups.append(int(Y_test[i][4]))
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
submission['accuracy_group'] = prediction_groups
print(submission)
submission.to_csv('submission.csv', index=False)